[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rhnvrm/mini-projects/blob/master/DNST_CIFAR_87_01.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 250
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 86s 1us/step


In [0]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator() 

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2, separable=False):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        if separable:
            Conv2D_3_3 = SeparableConv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        else:
            Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 16
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

num_filter = 32
First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

num_filter = 64
Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate, separable=False)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

num_filter = 64
Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate, separable=True)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

num_filter = 32
Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate, separable=True)
Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

num_filter = 16
Last_Block = add_denseblock(Fourth_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [19]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_116 (Conv2D)             (None, 32, 32, 16)   432         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_271 (BatchN (None, 32, 32, 16)   64          conv2d_116[0][0]                 
__________________________________________________________________________________________________
activation_271 (Activation)     (None, 32, 32, 16)   0           batch_normalization_271[0][0]    
__________________________________________________________________________________________________
conv2d_117

In [0]:
# determine Loss function and Optimizer
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [28]:
# alternative to model.fit_generator
# src: https://stackoverflow.com/questions/48820093/how-to-incorporate-validation-data-in-kerass-datagen-flow
# for e in range(epochs):
#     print('================')
#     print('| Epoch    ', e, ' |')
#     print('================')
#     batches = 0

#     # combine both generators, in python 3 using zip()
#     for (x_batch, y_batch), (val_x, val_y) in zip(
#                                  tr_datagen.flow(x_train, y_train, batch_size=batch_size),
#                                  val_datagen.flow(x_test, y_test, batch_size=batch_size)):
#         model.fit(x_batch, y_batch, validation_data=(val_x, val_y))
#         batches += 1
#         if batches >= len(x_train) / batch_size:
#             # we need to break the loop by hand because
#             # the generator loops indefinitely
#             break

filepath="file_name-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.fit_generator(
    train_datagen.flow(x_train, y_train),
    steps_per_epoch = len(x_train) / batch_size,
    epochs = epochs,
    validation_data = validation_datagen.flow(x_test, y_test),
    validation_steps = len(x_test) / batch_size,
    callbacks = callbacks_list,
    use_multiprocessing=True,
    workers=16
)


Epoch 1/250
391/390 [==============================] - 118s 303ms/step - loss: 1.9008 - acc: 0.2915 - val_loss: 1.7308 - val_acc: 0.3517

Epoch 00001: val_acc improved from -inf to 0.35166, saving model to file_name-01-0.35.hdf5
Epoch 2/250
391/390 [==============================] - 97s 249ms/step - loss: 1.6080 - acc: 0.4062 - val_loss: 1.9893 - val_acc: 0.3299

Epoch 00002: val_acc did not improve from 0.35166
Epoch 3/250
391/390 [==============================] - 98s 251ms/step - loss: 1.4706 - acc: 0.4617 - val_loss: 2.0973 - val_acc: 0.3956

Epoch 00003: val_acc improved from 0.35166 to 0.39557, saving model to file_name-03-0.40.hdf5
Epoch 4/250
391/390 [==============================] - 99s 254ms/step - loss: 1.4100 - acc: 0.4884 - val_loss: 1.8381 - val_acc: 0.4165

Epoch 00004: val_acc improved from 0.39557 to 0.41653, saving model to file_name-04-0.42.hdf5
Epoch 5/250
391/390 [==============================] - 98s 250ms/step - loss: 1.3258 - acc: 0.5225 - val_loss: 2.0565 - va

In [29]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 21s 2ms/step
Test loss: 0.4532780726671219
Test accuracy: 0.8701


In [30]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')